# Scientific Communication Workshop

# 1. Data simulation

In the following section, we will simulate an experiment where we image the fluorescence intensity oscillation of a set of cells induced by an external factor. The generated data will be the recovered for further analysis.

###**1.1. Setting up the environment**

### Install dependencies

In [ ]:
!pip install pygame -q

### Import packages

In [ ]:
import os
import numpy as np
from matplotlib import rc
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.animation as animation
import pygame
import random
import math
from IPython.display import HTML
from base64 import b64encode
import csv
import pandas as pd
import ipywidgets as widgets
from scipy import signal
from scipy import fftpack
import seaborn as sn

###Setting up Matplotlib animation parameters

The following is important for section *1.4. Visualizing the simulation*. It allows embedding Matplotlib animations into the Jupyter Notebook, in this case specifically, as an interactive JavaScript widget.

In [ ]:
rc('animation', html='jshtml')

### **1.2. Defining functions for the simulation**

In [ ]:
# Fool system to think it has a video card
os.environ['SDL_VIDEODRIVER']='dummy' 

pygame.init()
random.seed(801)

Width,Height = 1200,800
black = (0,0,0)

TotalFrames = 1000
NumBalls = 100
BallMinSize = 10
BallMaxSize = 40
AssociatingFactor = 70
MasterBallList = []


class OscillatingBalls(object):
    "Generates Oscillating Balls on the Screen of various size and oscillating frequency"
    
    def __init__(self):
        "Size is inversely proportional to the oscillation factor"
        self.Ballsize = random.randint(BallMinSize,BallMaxSize)
        self.Frequency = AssociatingFactor - self.Ballsize + random.randrange(-100,100,1)/50
        self.color = 170
        
    def BallPosition(self,x,y):
        self.Position = (x,y)
        
    def Oscillate(self,Time):
        self.color = 170 + int(45*math.sin((self.Frequency/180)*Time) ) + random.randint(-10, 10)

    def get_color(self):
        return (self.color)
     
        
def PopulateBalls(NumBalls):

    # Creates new csv files with the following names
    open("x_position.csv", "w")
    open("y_position.csv", "w")
    open("size.csv", "w")
    open("distance_to_tablet.csv","w")
    
    while len(MasterBallList)<NumBalls:
        Ball = OscillatingBalls()
        x = random.randint(50,1150)
        y = random.randint(50,800)
        Ball.BallPosition(x,y)
        Overlap = False

        if len(MasterBallList) > 0:
            for j in MasterBallList:
                Distance = math.sqrt((Ball.Position[0]-j.Position[0])**2 
                                     + (Ball.Position[1]-j.Position[1])**2)
                Intersection = Ball.Ballsize + j.Ballsize
                if Distance < Intersection:
                    Overlap = True
            if not Overlap:
                MasterBallList.append(Ball)
        elif len(MasterBallList) == 0:
            MasterBallList.append(Ball)
        if not Overlap:
          
          # Writes the x and y position and size of each ball in the corresponding csv file
          f0 = open("x_position.csv", "a")
          f0.write(str(Ball.Position[0]) + "\r\n")

          f1 = open("y_position.csv", "a")
          f1.write(str(Ball.Position[1]) + "\r\n")

          f2 = open("size.csv", "a")
          f2.write(str(Ball.Ballsize) + "\r\n")

          f3 = open("distance_to_tablet.csv","a")
          f3.write(str(round(math.sqrt((Ball.Position[0]-25)**2 + (Ball.Position[1]-25)**2),2)) + "\r\n")
    

def Simulate(output_dir="video_frames"):
    clock = pygame.time.Clock()
    fps = 10
    run = True
       
    PopulateBalls(NumBalls)

    # Create a csv file that will store the variable color for each ball throughout time
    open("color.csv", "w")

    for timepoint in range(TotalFrames):
        gameDisplay.fill(black)

        for Ball in MasterBallList:
            pygame.draw.circle(gameDisplay,(0,255,0),(25,25),25)  #Tablet of ligand
            pygame.draw.circle(gameDisplay,(Ball.color,Ball.color,Ball.color),Ball.Position,Ball.Ballsize)
            Ball.Oscillate(timepoint)

            # Each column will represent a different ball
            f = open("intensity.csv", "a")
            f.write(str(Ball.color) + ",")
            f.close()
        
        # Each row represents one timepoint
        f = open("intensity.csv", "a")
        f.write("\r\n")
        f.close()

        for event in pygame.event.get():
            if event.type == pygame.QUIT or not run:
                pygame.quit()

                
        filename = f"./{output_dir}/simulation_{timepoint}.png"
        pygame.image.save( gameDisplay, filename )
        pygame.display.update()
        clock.tick(fps)
    pygame.quit()

### **1.3. Generating video frames**

In [ ]:
gameDisplay = pygame.display.set_mode((Width,Height))
gameDisplay.fill(black)
!rm -r video_frames
!mkdir video_frames # Creates a folder to store frames
Simulate()

In [ ]:
fig = plt.figure(figsize=(8,6))
ax = plt.gca()
img = mpimg.imread('video_frames/simulation_1.png')
image = ax.imshow(img)

def frame(w):
    img = mpimg.imread(f'video_frames/simulation_{w}.png')
    image.set_data(img)
    return

In [ ]:
anim = animation.FuncAnimation(fig, frame, frames=200, blit=False, repeat=True,
                               interval=50)

### **1.4. Visualizing the simulation**

In [ ]:
# Takes about 1 min 20 sec to load
anim

# 2. Data Visualization and Analysis

###**2.1. Writing the relevant data extracted from the simulation into dataframes**

A DataFrame is a data structure that organizes data into a 2D table, like a spreadsheet.

In [ ]:
intensity_df = pd.read_csv('/content/intensity.csv', header=None).iloc[: , :-1]
xpos_df = pd.read_csv('/content/x_position.csv', header=None)
ypos_df = pd.read_csv('/content/y_position.csv', header=None)
size_df = pd.read_csv('/content/size.csv', header=None)
distance_df = pd.read_csv('/content/distance_to_tablet.csv',header =None)

#Change the column names
ypos_df.columns=['y_pos']
xpos_df.columns =['x_pos']
size_df.columns=['size']
distance_df.columns=['distance']

###**2.2. Visualizing the data**



In the first place, we will transpose the color matrix, so that the columns would represent the time and each row corresponds to a ball.

Therefore, we may concatenate the size and position Dataframes into the color DataFrame, so that the new one has a column for size, a column for x position and a column for y position.

In [ ]:
df = pd.concat([intensity_df.T, size_df, xpos_df, ypos_df, distance_df], axis=1)
df.head()

Afterwards, we will analize the variation of the intensity over time for each ball

In [ ]:
df.iloc[:,0:1000].T.describe()

Additionally, we may plor the variation of this intensity of the signal and even apply a tunable bandpass filter to remove the noise in the signal.

To create an interactive widget wherein the user selects the parameters of the filter and the ball number that is being represented, we use *ipywidgets*

In [ ]:
def plot_intensity(N):
    """
    Plots the intensity output of an specific ball (Ball number N) in time for the first 200 seconds
    """    
    plt.plot(df[N][0:200])
    #fig = px.line(color_df[N][0:200])
    #fig.show()

def bandpass_filter(ball_number, N, Wn):
    """
    Return a filtered signal
    """    
    y = intensity_df[ball_number].iloc[0:200]
    b, a = signal.butter(N, [Wn[0], Wn[1]], 'bandpass')
    output_signal = signal.filtfilt(b, a, y)
    plt.plot(output_signal)

In [ ]:
widgets.interact(bandpass_filter, ball_number = widgets.IntSlider(value=0, min=0, max=99,  description='Ball Number'),
                 N = widgets.IntSlider(value=1, min=1, max=4,  description='Filter stringency'),
                 Wn = widgets.FloatRangeSlider(value=[0, 0.999], min=0.001,max=0.999,step=0.001))

We can get the oscilation frequency of the balls applying a fourier transform to get the main frequency of the intensity signals

In [ ]:
def get_main_frequency (array):
  fourier = fftpack.fft((array - np.mean(array)).values)
  frequencies = fftpack.fftfreq(array.size)
  freq = frequencies[np.argmax(np.abs(fourier))]
  return round(freq*2*np.pi,2) # we multiply by 2*pi so that it is consistent with our initial simulation input

In [ ]:
df['main_frequency'] = df.loc[:,0:999].apply(get_main_frequency, axis=1)
df.head()

Finally, we may create a final curated DataFrame with only the relevant variables to be analyzed:

*   size
*   x and y position
*   distance to the tablet
*   oscillation frequency



In [ ]:
final_df = df[['size','y_pos','x_pos','distance','main_frequency']]

###**2.3. Analyzing the correlation between variables**

In [ ]:
pd.plotting.scatter_matrix(final_df, figsize=(10,10))
plt.show()

In [ ]:
corrM = final_df.corr()
sn.heatmap(corrM, annot=True, cmap='vlag')

In [ ]:
final_df['main_frequency'].corr(final_df['size'])

In [ ]:
plt.scatter(final_df['size'], final_df['main_frequency'])
plt.xlabel("Size")
plt.ylabel("Oscillation Frequency")
plt.title("Variation of the oscillation frequency of the intensity of the balls with their size")
plt.show()